# Context management

In this notebook we go through some features of Rasa. It is commonly used for retrieval-based chatbots. We cover in this part the following topics:

- intent and entities understanding,
- the learning process of intent understanding,
- state machine as a crucial part of chatbot conversation,
- context management with Rasa core.

## Intent understanding

In this section we use Rasa to build a very simple HR assistant bot. We can use Rasa as a server or use it directly from Python level. To start Rasa server you need to execute the following command:
```python3 -m rasa_nlu.server --path projects &```
It starts a server on default port 5000. You can test it using the request package. We should get the intent of the phrase `hi`.

![](images/message_3.png)

In [5]:
import requests

def get_intent(sentence):
    url = "http://localhost:5000/parse"
    payload = {"q":sentence}
    response = requests.get(url,params=payload)    
    print(response.json())
    intent = response.json()['intent']
    if intent['confidence'] > 0.5: 
        return intent['name']
    return response.json()

get_intent("hi")

{'intent': {'name': 'greet', 'confidence': 1.0}, 'entities': [], 'text': 'hi', 'project': 'default', 'model': 'fallback'}


'greet'

In [6]:
get_intent("hello")

{'intent': {'name': 'greet', 'confidence': 1.0}, 'entities': [], 'text': 'hello', 'project': 'default', 'model': 'fallback'}


'greet'

To use Rasa from Python level you need to prepare a config file that contains the pipeline and the filename of examples used for learning.

In [46]:
co = """
{
  "pipeline": "spacy_sklearn",
  "path" : ".",
  "data" : ".anna.json"
}
"""

config_file = open("config.json", "w")
config_file.write(co)
config_file.close()

The data file contains examples that are used for training.

In [7]:
anna_common_examples = """
{
  "rasa_nlu_data": {
    "entity_synonyms": [
      {
        "value": "candidate",
        "synonyms": ["developer", "data scientist"]
      }
    ],
    "common_examples": [
      {
        "text": "hey", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "howdy", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hey there",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hello", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hi", 
        "intent": "greet", 
        "entities": []
      },
      {
        "text": "good morning",
        "intent": "greet",
        "entities": []
      },
      {
        "text": "good evening",
        "intent": "greet",
        "entities": []
      },
      {
        "text": "dear sir",
        "intent": "greet",
        "entities": []
      },
      {
        "text": "yes", 
        "intent": "affirm", 
        "entities": []
      }, 
      {
        "text": "yep", 
        "intent": "affirm", 
        "entities": []
      }, 
      {
        "text": "yeah", 
        "intent": "affirm", 
        "entities": []
      },
      {
        "text": "indeed",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "that's right",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "ok",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "great",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "right, thank you",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "add candidate",
        "intent": "candidate_add",
        "entities": [
            {
      "start": 5,
      "end": 13,
      "value": "candidate",
      "entity": "candidate"
        }
        ]
      },         
      {
        "text": "adding candidate",
        "intent": "candidate_add",
        "entities": [
            {
              "start": 8,
              "end": 16,
              "value": "candidate",
              "entity": "candidate"
            }        
        ]
      },
      {
        "text": "please add candidate",
        "intent": "candidate_add",
        "entities": []
      },              
      {
        "text": "please add new candidate",
        "intent": "candidate_add",
        "entities": []
      },           
      {
        "text": "we have new prescreening upcoming",
        "intent": "candidate_add",
        "entities": []
      }, 
      {
        "text": "we have a new candidate for prescreening",
        "intent": "candidate_add",
        "entities": []
      },         
      {
        "text": "correct",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "great choice",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "sounds really good",
        "intent": "affirm",
        "entities": []
      },
      {
        "text": "bye", 
        "intent": "goodbye", 
        "entities": []
      }, 
      {
        "text": "goodbye", 
        "intent": "goodbye", 
        "entities": []
      }, 
      {
        "text": "good bye", 
        "intent": "goodbye", 
        "entities": []
      }, 
      {
        "text": "stop", 
        "intent": "goodbye", 
        "entities": []
      }, 
      {
        "text": "end", 
        "intent": "goodbye", 
        "entities": []
      },
      {
        "text": "farewell",
        "intent": "goodbye",
        "entities": []
      },
      {
        "text": "Bye bye",
        "intent": "goodbye",
        "entities": []
      },
      {
        "text": "have a good one",
        "intent": "goodbye",
        "entities": []
      }
    ]
  }
}
"""

training_data = open("anna.json", "w")
training_data.write(anna_common_examples)
training_data.close()

The training is straight forward.

In [ ]:
#pip3 install -U scikit-learn==0.19.1

In [8]:
from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer, Interpreter
from rasa_nlu.components import ComponentBuilder
import rasa_nlu.config

cfg = 'config.json'
training_data = load_data('anna.json')
trainer = Trainer(rasa_nlu.config.load(cfg))
trainer.train(training_data)
model_directory = trainer.persist('.')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


To get the intent we use the parse method.

In [9]:
from rasa_nlu.model import Metadata, Interpreter

interpreter = Interpreter.load(model_directory)

interpreter.parse(u"yes")

{'entities': [],
 'intent': {'confidence': 0.6837588947986979, 'name': 'affirm'},
 'intent_ranking': [{'confidence': 0.6837588947986979, 'name': 'affirm'},
  {'confidence': 0.2560145833911606, 'name': 'greet'},
  {'confidence': 0.042431731595500566, 'name': 'goodbye'},
  {'confidence': 0.017794790214640692, 'name': 'candidate_add'}],
 'text': 'yes'}

### How does it work?

The training is divided into several parts as shown below.

![](images/classifier.png)

#### Exercise 1. Train Rasa to discover new intent

Extend the training examples and add an intent `change_status` with entities: `passed` and `failed`.

In [10]:
anna_common_examples = 
"""

""""
training_data = open("anna_new.json", "w")
training_data.write(anna_common_examples)
training_data.close()

Train it:

In [11]:
from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer, Interpreter
from rasa_nlu.components import ComponentBuilder
import rasa_nlu.config

cfg = 'config.json'
training_data = load_data('anna_new.json')
trainer = Trainer(rasa_nlu.config.load(cfg))
trainer.train(training_data)
model_directory = trainer.persist('.')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


Test it:

In [12]:
from rasa_nlu.model import Metadata, Interpreter

interpreter = Interpreter.load(model_directory)

interpreter.parse(u"the developer didn't passed")

{'entities': [],
 'intent': {'confidence': 0.6204533422709783, 'name': 'change_status'},
 'intent_ranking': [{'confidence': 0.6204533422709783,
   'name': 'change_status'},
  {'confidence': 0.13981978300841671, 'name': 'affirm'},
  {'confidence': 0.09130984094477532, 'name': 'candidate_add'},
  {'confidence': 0.07576837710026818, 'name': 'goodbye'},
  {'confidence': 0.07264865667556153, 'name': 'greet'}],
 'text': "the developer didn't passed"}

## Building stories with Rasa

To build a chatbot with Rasa that has a focus on the context management, we need to build the dataset of intents as before, but also stories. 


![](images/intent_entities.png)

Rasa core for building stories need a bit more configuration than in the previous example. We need to setup the following:
- the configuration of language and machine learning backend,
- setup the domain with sample chatbot responses,
- define the stories.
After this step we need to train Rasa, but we still need feed it with intents after it.

A basic configuration for Rasa is needed like to language and pipeline. The pipeline defines the way how we want to train our dataset.

In [13]:
rasa_config = """
policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
  - name: FormPolicy
"""
%store rasa_config > rasa_config.yml

Writing 'rasa_config' (str) to file 'rasa_config.yml'.


We need to define a few stories for our chatbot:

In [14]:
stories_md = """
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye
"""
%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


We need to set the domain:

In [15]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""
%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [16]:
!python3 -m rasa_core.train -c rasa_config.yml -d domain.yml -s stories.md -o models/krakow

/usr/local/lib/python3.6/dist-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|#####| 4/4 [00:00<00:00, 109.59it/s, # trackers=14]
2018-11-22 11:38:44.655192: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 15)             0         
_________________________________________________________________
lstm (LST

62/62 [==============================] - 0s 244us/step - loss: 1.4345 - acc: 0.4355
Epoch 70/100
62/62 [==============================] - 0s 153us/step - loss: 1.4449 - acc: 0.4516
Epoch 71/100
62/62 [==============================] - 0s 137us/step - loss: 1.4175 - acc: 0.4516
Epoch 72/100
62/62 [==============================] - 0s 120us/step - loss: 1.3996 - acc: 0.4516
Epoch 73/100
62/62 [==============================] - 0s 161us/step - loss: 1.4121 - acc: 0.4355
Epoch 74/100
62/62 [==============================] - 0s 255us/step - loss: 1.4241 - acc: 0.4194
Epoch 75/100
62/62 [==============================] - 0s 120us/step - loss: 1.4044 - acc: 0.4516
Epoch 76/100
62/62 [==============================] - 0s 154us/step - loss: 1.3885 - acc: 0.4516
Epoch 77/100
62/62 [==============================] - 0s 112us/step - loss: 1.3687 - acc: 0.4516
Epoch 78/100
62/62 [==============================] - 0s 161us/step - loss: 1.4050 - acc: 0.4355
Epoch 79/100
62/62 [=======================

In [17]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad
"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [18]:
nlu_config = """
language: en
pipeline: tensorflow_embedding
"""
%store nlu_config > nlu_config.yml

Writing 'nlu_config' (str) to file 'nlu_config.yml'.


In [19]:
!python3 -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project lublin --verbose

2018-11-22 11:40:46 INFO     rasa_nlu.training_data.loading  - Training data format of nlu.md is md
2018-11-22 11:40:46 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 39 (6 distinct intents)
	- Found intents: 'greet', 'mood_great', 'mood_deny', 'mood_affirm', 'mood_unhappy', 'goodbye'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

2018-11-22 11:40:46 INFO     rasa_nlu.model  - Starting to train component tokenizer_whitespace
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Finished training component.
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Starting to train component ner_crf
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Finished training component.
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Starting to train component ner_synonyms
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Finished training component.
2018-11-22 11:40:46 INFO     rasa_nlu.model  - Starting to train component intent_featurizer_count_vectors
201

In [20]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

interpreter = RasaNLUInterpreter('models/lublin/nlu')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/lublin', interpreter=interpreter)

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        messages.append(r.get("text"))

stop


### Exercise 2: Add a menu story for a restaurant

You should be able to display the menu depending on the part of the menu like: starters, soups, main dishses, desserts and drinks.

In [ ]:
# put your stories here

In [ ]:
# put your domain here

In [ ]:
!python3 -m rasa_core.train -d domain.yml -s stories.md -o models/restaurant

In [ ]:
# put your intentes here

In [ ]:
!python3 -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project restaurant --verbose

Test it:

In [ ]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

interpreter = RasaNLUInterpreter('models/restaurant/nlu')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/restaurant', interpreter=interpreter)

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        messages.append(r.get("text"))